In [ ]:
# Выгрузка библиотек для работы с данными и анализа
import pandas as pd
import numpy as np
import seaborn as sns

from matplotlib import pyplot as plt

In [ ]:
# Выгружаем датафреймы
df = pd.read_csv('train.csv')
df_valid = pd.read_csv('valid.csv')

# Команда для отображения всех колонок в датафреймах
pd.set_option('display.max_columns', None)

In [ ]:
# Удаление колонок при 30% пропусков
columns_to_drop = []

for col in df.columns:
    if df[col].isna().sum() >= len(df[col])*0.3:
        columns_to_drop.append(col)
    
    if df_valid[col].isna().sum() >= len(df_valid[col])*0.3:
        columns_to_drop.append(col)

columns_to_drop = list(set(columns_to_drop))

df.drop(columns = columns_to_drop, inplace = True)
df_valid.drop(columns = columns_to_drop, inplace = True)

In [ ]:
# Удаление колонки id
df.drop('col1454', axis=1, inplace=True)
df_valid.drop('col1454', axis=1, inplace=True)

In [ ]:
# Импорт необходимых библиотек для построение модели
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer  # Импорт для обработки пропущенных значений
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, classification_report
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline  # Для использования SMOTE в пайплайне

In [ ]:
# Разделение данных на признаки и целевую переменную
X = df.drop('target', axis=1)
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

X_valid = df_valid.drop('target', axis=1)
y_valid = df_valid['target']

In [ ]:
# Преобразуем столбец 'report_date' в числовой формат
def convert_dates(X):
    if 'report_date' in X.columns:
        X['report_date'] = pd.to_datetime(X['report_date']).astype(int) / 10**9  # Преобразуем в секунды с эпохи
    return X

# Применяем функцию к данным
X_train = convert_dates(X_train)
X_test = convert_dates(X_test)
X_valid = convert_dates(X_valid)

# Определение числовых и категориальных столбцов
numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X_train.select_dtypes(include=['object', 'category']).columns

# Препроцессинг данных
# Добавляем SimpleImputer для обработки пропущенных значений
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Заполнение пропущенных значений медианой
    ('scaler', StandardScaler())  # Стандартизация числовых признаков
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Заполнение пропущенных значений наиболее часто встречающимся значением
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # One-Hot Encoding для категориальных признаков
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

In [ ]:
# # Создание пайплайна с SMOTE и классификатором
# pipeline = ImbPipeline(steps=[
#     ('preprocessor', preprocessor),  # Препроцессинг данных
#     ('smote', SMOTE(random_state=42)),  # Балансировка классов с использованием SMOTE
#     ('classifier', RandomForestClassifier(random_state=0))  # Классификатор
# ])

# # Определение сетки гиперпараметров
# param_grid = {
#     'classifier__n_estimators': [100, 200, 300],
#     'classifier__max_depth': [None, 10, 20, 30],
#     'classifier__min_samples_split': [2, 5, 10],
#     'classifier__min_samples_leaf': [1, 2, 4],
#     'classifier__max_features': ['sqrt', 'log2', None]
# }

# # Поиск гиперпараметров с использованием GridSearchCV
# grid_search = GridSearchCV(pipeline, param_grid, scoring='roc_auc', cv=3, n_jobs=-1)
# grid_search.fit(X_train, y_train)

# # Лучшие гиперпараметры
# best_params = grid_search.best_params_
# print("Best parameters found:", best_params)
# # >> {'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}

In [ ]:
# # Обучение модели с лучшими гиперпараметрами
# best_rf = grid_search.best_estimator_

# # Оценка модели на тестовых данных
# y_pred = best_rf.predict(X_test)
# y_pred_proba = best_rf.predict_proba(X_test)[:, 1]

In [ ]:
best_params = {'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}

# Создание пайплайна с SMOTE и классификатором
best_rf = ImbPipeline(steps=[
    ('preprocessor', preprocessor),  # Препроцессинг данных
    ('smote', SMOTE(random_state=42)),  # Балансировка классов с использованием SMOTE
    ('classifier', RandomForestClassifier(random_state=0, **best_params))  # Классификатор
])

# Обучение модели с лучшими гиперпараметрами
best_rf.fit(X_train, y_train)

In [ ]:
# Оценка модели на тестовых данных
y_pred = best_rf.predict(X_test)
y_pred_proba = best_rf.predict_proba(X_test)[:, 1]

# Вывод метрик
print("Точность (Accuracy):", accuracy_score(y_test, y_pred))
print("Матрица ошибок (Confusion Matrix):\n", confusion_matrix(y_test, y_pred))
print("Точность (Precision):", precision_score(y_test, y_pred, average='weighted', zero_division=0))
print("Полнота (Recall):", recall_score(y_test, y_pred, average='weighted'))
print("F1-мера (F1-score):", f1_score(y_test, y_pred, average='weighted'))
print("ROC-AUC (для бинарной классификации):", roc_auc_score(y_test, y_pred_proba))
print("Отчет о классификации:\n", classification_report(y_test, y_pred, zero_division=0))

In [ ]:
# Оценка модели на валидационных данных
y_pred_v = best_rf.predict(X_valid)
y_pred_proba_v = best_rf.predict_proba(X_valid)[:, 1]

# Вывод метрик
print("Точность (Accuracy):", accuracy_score(y_valid, y_pred_v))
print("Матрица ошибок (Confusion Matrix):\n", confusion_matrix(y_valid, y_pred_v))
print("Точность (Precision):", precision_score(y_valid, y_pred_v, average='weighted', zero_division=0))
print("Полнота (Recall):", recall_score(y_valid, y_pred_v, average='weighted'))
print("F1-мера (F1-score):", f1_score(y_valid, y_pred_v, average='weighted'))
print("ROC-AUC (для бинарной классификации):", roc_auc_score(y_valid, y_pred_proba_v))
print("Отчет о классификации:\n", classification_report(y_valid, y_pred_v, zero_division=0))